In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit

In [ ]:
# import requests, pathlib

# url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
# local_path = pathlib.Path("Sample.db")

# if local_path.exists():
#     print(f"{local_path} already exists, skipping download.")
# else:
#     response = requests.get(url)
#     if response.status_code == 200:
#         local_path.write_bytes(response.content)
#         print(f"File downloaded and saved as {local_path}")
#     else:
#         print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as Sample.db


In [11]:
# file = pathlib.Path("Sample.db")
# print(file.stat().st_size)

1007616


In [2]:
load_dotenv()

os.environ["LANGSMITH_TRACING"] = 'true'
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [3]:
model = init_chat_model(model='llama-3.3-70b-versatile', temperature=0.3, model_provider='groq')

In [12]:
db = SQLDatabase.from_uri("sqlite:///Sample.db")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM Artist LIMIT 5;")}')

Dialect: sqlite
Available tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']
Sample output: [(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains')]


In [14]:
sql_toolkit = SQLDatabaseToolkit(db=db, llm=model)

query_tools = sql_toolkit.get_tools()

for tool in query_tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [ ]:
system_prompt = """
                You are an agent designed to interact with a SQL database.
                Given an input question, create a syntactically correct {dialect} query to run,
                then look at the results of the query and return the answer. Unless the user
                specifies a specific number of examples they wish to obtain, always limit your
                query to at most {top_k} results.
                You can order the results by a relevant column to return the most interesting
                examples in the database. Never query for all the columns from a specific table,
                only ask for the relevant columns given the question.
                You MUST double check your query before executing it. If you get an error while
                executing a query, rewrite the query and try again.
                DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the
                database. To start you should ALWAYS look at the tables in the database to see 
                what you can query. Do NOT skip this step.
                Then you should query the schema of the most relevant tables.
                """.format(dialect=db.dialect,top_k=5,)

In [16]:
agent = create_agent(
    model,
    query_tools,
    system_prompt=system_prompt,
)

In [17]:
question = "Which genre on average has the longest tracks?"

In [18]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which genre on average has the longest tracks?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (8w5819rw1)
 Call ID: 8w5819rw1
  Args:
  sql_db_schema (hmhyjm0g5)
 Call ID: hmhyjm0g5
  Args:
    table_names: tracks, genres
  sql_db_query_checker (wmzdrsvqs)
 Call ID: wmzdrsvqs
  Args:
    query: SELECT genres.name, AVG(tracks.duration) as avg_duration FROM tracks JOIN genres ON tracks.genre_id = genres.id GROUP BY genres.name ORDER BY avg_duration DESC LIMIT 5
  sql_db_query (8pa4yr6ja)
 Call ID: 8pa4yr6ja
  Args:
    query: SELECT genres.name, AVG(tracks.duration) as avg_duration FROM tracks JOIN genres ON tracks.genre_id = genres.id GROUP BY genres.name ORDER BY avg_duration DESC LIMIT 5
================================= Tool Message =================================
Name: sql_db_query

Error: (sqlite3.OperationalError) no such table: tra